EECS 4412 Project - Phase II
Maryam Salarian
Analysis of the dataset + preprocessing + ....

In [4]:
# 1. load the dataset

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
reviews = pd.read_csv("/content/drive/MyDrive/EECS4412/data/project/Books_rating.csv")

Mounted at /content/drive


In [ ]:
# 2. display the dataset to get an idea

reviews.head(10)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
5,0826414346,Dr. Seuss: American Icon,NaN,A2F6NONFUDB6UK,Malvin,2/2,4.0,1127174400,One of America's greatest creative talents,"""Dr. Seuss: American Icon"" by Philip Nel is a ..."
6,0826414346,Dr. Seuss: American Icon,NaN,A14OJS0VWMOSWO,Midwest Book Review,3/4,5.0,1100131200,A memorably excellent survey of Dr. Seuss' man...,Theodor Seuss Giesel was best known as 'Dr. Se...
7,0826414346,Dr. Seuss: American Icon,NaN,A2RSSXTDZDUSH4,J. Squire,0/0,5.0,1231200000,Academia At It's Best,When I recieved this book as a gift for Christ...
8,0826414346,Dr. Seuss: American Icon,NaN,A25MD5I2GUIW6W,"J. P. HIGBED ""big fellow""",0/0,5.0,1209859200,And to think that I read it on the tram!,Trams (or any public transport) are not usuall...
9,0826414346,Dr. Seuss: American Icon,NaN,A3VA4XFS5WNJO3,Donald Burnside,3/5,4.0,1076371200,Fascinating account of a genius at work,"As far as I am aware, this is the first book-l..."


In [ ]:
# 3. as displayed, some of the books have multiple reviews and ratings
# handling approach: treat each review as a separate sample, and once sentiment assigned, take majority vote
# alternative to majority vote: take weighted sentiment, using review/helpfulness as weight, and assigning small weight to those with review/helpfulness = 0.
# size of the unaltered dataset
print("size of unaltered reviews dataset:", reviews.shape)

# checking data types, as non-numeric data type for rating will affect preprocessing
print ("\ndata type of attributes:\n", reviews.dtypes)

# rename relevant attrs for ease of reference: Title to title, review/score to rating, review/helpfulness to helpfulness, review/text to review
reviews.rename(columns={'Title':'title', 'review/score': 'rating', 'review/helpfulness': 'helpfulness', 'review/text':'review'}, inplace=True)

# drop all other cols
columns_to_keep = ["title", "rating", "helpfulness", "review"]
reviews = reviews[columns_to_keep]
print("size of dataset after dropping 6 out of 10 attributes:", reviews.shape)

# check the max and min value for rating (review/score) in the dataset
print("\nmin rating:", reviews["rating"].min())
print("max rating:", reviews["rating"].max())


size of unaltered reviews dataset: (3000000, 10)

data type of attributes:
 Id                     object
Title                  object
Price                 float64
User_id                object
profileName            object
review/helpfulness     object
review/score          float64
review/time             int64
review/summary         object
review/text            object
dtype: object
size of dataset after dropping 6 out of 10 attributes: (3000000, 4)

min rating: 1.0
max rating: 5.0


In [ ]:
# 4. add new col, sentiment, based on review rating
# will be added to the entire dataset as target attribute
# rating >= 4 is positive: +1
# rating =< 2 is negative: -1
# rating =3 is netural: 0

# perform rating-specific preprocessing:

# a) convert rating to numreic -> from above data types we conclude that rating is already in numeric format: float64
# b) remove rows with missing or NaN rating
reviews = reviews.dropna(subset=["rating"])
# c) ensure all ratings are within range -> from above rating range, we confirm that all ratings are within [1,5]

# generate the new sentiment column
def assign_sentiment(x):
    if x >= 4:
        return 1
    elif x == 3:
        return 0
    else:
        return -1

# apply the above function to every value in the rating col, and store result in new col
reviews["sentiment"] = reviews["rating"].apply(assign_sentiment)

# convert helpfulness from object to numeric values, fill NaN with 0
def fraction_to_float(x):
    if isinstance(x, str) and "/" in x:
        try:
            num, denom = x.split("/")
            return float(num) / float(denom)
        except:
            return np.nan  # invalid fraction
    else:
        # try to convert directly to float
        return pd.to_numeric(x, errors='coerce')
reviews["helpfulness"] = reviews["helpfulness"].apply(fraction_to_float)
reviews["helpfulness"] = reviews["helpfulness"].fillna(0)

print("size of dataset after removing NaN ratings and adding new col:", reviews.shape)
reviews.head(5)


size of dataset after removing NaN ratings and adding new col: (3000000, 5)


,title,rating,helpfulness,review,sentiment
0,Its Only Art If Its Well Hung!,4.0,1.000000,This is only for Julie Strain fans. It's a col...,1
1,Dr. Seuss: American Icon,5.0,1.000000,I don't care much for Dr. Seuss but after read...,1
2,Dr. Seuss: American Icon,5.0,0.909091,"If people become the books they read and if ""t...",1
3,Dr. Seuss: American Icon,4.0,1.000000,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",1
4,Dr. Seuss: American Icon,4.0,1.000000,Philip Nel - Dr. Seuss: American IconThis is b...,1


In [ ]:
pip install contractions

In [ ]:
# 5. perform further preprocessing on the dataset
import re

# a) remove rows with missig reviews: NaN, '', ""
# drop rows where review is NaN
reviews = reviews.dropna(subset=["review"])
# remove empty strings or whitespace-only reviews
df = reviews[reviews["review"].str.strip().astype(bool)]
print("size of dataset after removing NaN or empty reviews:", reviews.shape)

# b) remove identical rows with duplicate review per book
reviews = reviews.drop_duplicates(subset=["title", "review"])
print("size of dataset after dropping duplicates:", reviews.shape)

# c) replace english contractions with full form: won't -> will not
contraction_map = {
    "can't": "can not",
    "won't": "will not",
    "n't": " not",
    "'re": " are",
    "'s": " is",
    "'d": " would",
    "'ll": " will",
    "'t": " not",
    "'ve": " have",
    "'m": " am"
}
# pattern = re.compile("({})".format("|".join(map(re.escape, contraction_map.keys()))))
# def expand_contractions_fast(text: str) -> str:
#     if not isinstance(text, str):
#         return text
#     return pattern.sub(lambda m: contraction_map[m.group(0)], text)
# chunksize = 50000
# for start in range(0, len(reviews), chunksize):
#     end = start + chunksize
#     reviews.loc[start:end, "review"] = (
#         reviews.loc[start:end, "review"].apply(expand_contractions_fast)
#     )

def expand_contractions_vectorized(series):
    for k, v in contraction_map.items():
        series = series.str.replace(k, v, regex=True)
    return series

reviews["review"] = expand_contractions_vectorized(reviews["review"])
print("contraction expansion completed.")


size of dataset after removing NaN or empty reviews: (2999992, 5)
size of dataset after dropping duplicates: (2616740, 5)
contraction expansion completed.


In [ ]:
import re

# d) to prevent overfitting and reduce noise, remove reviews with < 2 words
reviews = reviews[reviews["review"].str.len() >= 2]
print("reviews with less than 2  words dropped.")

# e) lower case all characters in review
reviews["review"] = reviews["review"].str.lower()
print("all chars lowercased.")

# # f) remove leading and trailing white spaces + normalize space between chars
# reviews["review"] = reviews["review"].str.replace(r"\s+", " ", regex=True).str.strip()
# print("white spaces normalized.")

# # g) remove non-text/num chars, keep the spaces
# reviews["review"] = reviews["review"].str.replace(r"[^a-z0-9\s']", " ", regex=True)
# print("punctuation removal completed.")

clean_pattern = re.compile(r"[^a-z0-9\s']+")

def fast_clean(text: str) -> str:
    if not isinstance(text, str):
        return text
    # 1. Remove unwanted characters (punctuation, symbols)
    text = clean_pattern.sub(" ", text)
    # 2. Normalize whitespace (no need for regex here)
    text = " ".join(text.split())
    return text

chunksize = 50000
for start in range(0, len(reviews), chunksize):
    end = start + chunksize
    reviews.loc[start:end, "review"] = (
        reviews.loc[start:end, "review"].apply(fast_clean)
    )

reviews with less than 2  words dropped.
all chars lowercased.


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pickle
import tqdm

In [ ]:
# 6. perfrom text - order them appropriately
# updated logic to use PorterStemmer > deletef after laptop auto reboot
# downloaded the result to /content/drive/MyDrive/EECS4412/data/project/reviews_cleaned.csv

# tokenize
def tokenize(text):
    return token_pattern.findall(text.lower())

# remove stopwords
stop_words = set(stopwords.words('english'))
negation_words = {"not", "no", "never", "none"}
stop_words -= negation_words
def remove_stopwords(tokens):
    return [t for t in tokens if t not in stop_words]

# define stemmer and find word stems
stemmer = PorterStemmer()
def stem_tokens(tokens):
    return [stemmer.stem(t) for t in tokens]

# put all steps together
def preprocess_text(text):
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stem_tokens(tokens)
    # return string for TF-IDF
    return " ".join(tokens)


# put steps together:
reviews.to_csv("/content/drive/MyDrive/EECS4412/data/project/reviews_cleaned.csv", index=False)

with open("/content/reviews_cleaned.pkl", "wb") as f:
    pickle.dump(reviews, f)

reviews.head(5)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


KeyboardInterrupt: 

In [5]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
reviews_cleaned = pd.read_csv("/content/drive/MyDrive/EECS4412/data/project/reviews_cleaned.csv")
reviews_cleaned.head(5)

,title,rating,helpfulness,review,sentiment
0,Its Only Art If Its Well Hung!,4.0,1.000000,juli strain fan collect photo page worth nice ...,1
1,Dr. Seuss: American Icon,5.0,1.000000,not care much dr seuss read philip nel book ch...,1
2,Dr. Seuss: American Icon,5.0,0.909091,peopl becom book read child father man dr seus...,1
3,Dr. Seuss: American Icon,4.0,1.000000,theodor seuss geisel aka quot dr seuss quot on...,1
4,Dr. Seuss: American Icon,4.0,1.000000,philip nel dr seuss american iconthi basic aca...,1


In [34]:
reviews_cleaned = reviews_cleaned.dropna(subset=["review"]).reset_index(drop=True)
# reviews_cleaned.reset_index(drop=True, inplace=True)

reviews_cleaned.shape

(2616702, 5)

In [35]:
# remove NaNs and rewrite to the same file
reviews_cleaned.to_csv(
    "/content/drive/MyDrive/EECS4412/data/project/reviews_cleaned.csv",
    index=False
)
print("reviews_cleaned.csv overwritten with NaN-free version.")

reviews_cleaned.csv overwritten with NaN-free version.


In [7]:
print(reviews_cleaned.shape)
print(reviews_cleaned["review"].isna().sum())

(2616702, 5)
0


In [40]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
import joblib

DTYPE = np.float32
PROJECT_PATH = "/content/drive/MyDrive/EECS4412/data/project/"
NGRAM_RANGE = (1,2)
MAX_FEATURES = 150_000
SAMPLE_SIZE = 150_000

# sample 150k reviews to fit the vectorizer
sample = reviews_cleaned.sample(SAMPLE_SIZE, random_state=42)["review"].astype(str).tolist()

vectorizer = CountVectorizer(
    ngram_range=NGRAM_RANGE,
    binary=True,
    dtype=DTYPE,
    max_features=MAX_FEATURES
)

vectorizer.fit(sample)
vocab = vectorizer.get_feature_names_out()
print(f"Learned vocab size: {len(vocab)}")
joblib.dump(vectorizer, PROJECT_PATH + "vectorizer.pkl")

reviews_list = reviews_cleaned["review"].astype(str).tolist()
total_reviews = len(reviews_list)
print("total reviews:", total_reviews)


Learned vocab size: 150000
total reviews: 2616702


In [41]:
# stream DF counting in chunks
from scipy.sparse import csr_matrix

CHUNK_SIZE = 100_000

total_reviews = len(reviews_cleaned)
df_counts = np.zeros(len(vocab), dtype=DTYPE)

for start in tqdm(range(0, total_reviews, CHUNK_SIZE), desc="Counting DF"):
    end = min(start + CHUNK_SIZE, total_reviews)
    batch = reviews_cleaned.iloc[start:end]["review"].astype(str).tolist()

    X_chunk = vectorizer.transform(batch)  # sparse CSR matrix
    # sum over rows (axis=0) -> number of docs each token appears in
    df_chunk = np.array((X_chunk > 0).sum(axis=0)).ravel()
    df_counts += df_chunk

np.save(PROJECT_PATH + "df_counts.npy", df_counts)
print("Document frequency counting complete!")

Counting DF: 100%|██████████| 27/27 [07:24<00:00, 16.47s/it]

Document frequency counting complete!


In [42]:
# apply DF thresholds and reduce vocab

N = len(reviews_cleaned)
MIN_DF_PROP = 1e-4
MAX_DF_PROP = 0.95
MAX_KEEP = 50_000

min_df_cutoff = int(N * MIN_DF_PROP)
max_df_cutoff = int(N * MAX_DF_PROP)

# boolean array for vocabs
mask = (df_counts >= min_df_cutoff) & (df_counts <= max_df_cutoff)
indices_kept = np.where(mask)[0]

reduced_vocab = vocab[indices_kept]

# keep top MAX_KEEP by DF if necessary
if len(reduced_vocab) > MAX_KEEP:
    sorted_idx = np.argsort(df_counts[indices_kept])[::-1]
    top_idx = sorted_idx[:MAX_KEEP]
    indices_kept = indices_kept[top_idx]
    reduced_vocab = vocab[indices_kept]

np.save(PROJECT_PATH + "reduced_vocab.npy", reduced_vocab)
print("Reduced vocab size:", len(reduced_vocab))

Reduced vocab size: 50000


In [43]:
# build TF-IDF vectorizer with reduced vocab
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
import joblib

PROJECT_PATH = "/content/drive/MyDrive/EECS4412/data/project/"
NGRAM_RANGE = (1,2)
SAMPLE_SIZE = 150_000

# load reduced vocab
reduced_vocab = np.load(PROJECT_PATH + "reduced_vocab.npy", allow_pickle=True)

tfidf = TfidfVectorizer(
    ngram_range=NGRAM_RANGE,
    vocabulary={t: i for i, t in enumerate(reduced_vocab)},
    # reduce impact of very frequent words, used instead of raw frequency
    sublinear_tf=True,
    # L2 norm, prevents longer docs from dominating
    norm='l2',
    token_pattern=r"[a-zA-Z]+(?:'[a-z]+)?"
)

sample_texts = reviews_cleaned.sample(SAMPLE_SIZE, random_state=42)['review'].astype(str).tolist()
print("Sample text creted!")

tfidf.fit(sample_texts)
joblib.dump(tfidf, PROJECT_PATH + "tfidf_reduced_vocab.pkl")
print("TF-IDF computed on reduced vocab!")

Sample text creted!
TF-IDF computed on reduced vocab!


In [44]:
# apply TF-IDF to the dataset and save in chunks

from scipy.sparse import save_npz, vstack
from joblib import load

N = len(reviews_cleaned)
CHUNK_SIZE = 100_000

# load tfidf
tfidf = load(PROJECT_PATH + "tfidf_reduced_vocab.pkl")

tfidf_chunk_files = []

for start in tqdm(range(0, N, CHUNK_SIZE), desc="TF-IDF transform"):
    end = min(start + CHUNK_SIZE, N)
    batch = reviews_cleaned.iloc[start:end]['review'].astype(str).tolist()
    X_chunk = tfidf.transform(batch)
    fname = PROJECT_PATH + f"tfidf_chunk_{start}_{end}.npz"
    save_npz(fname, X_chunk.astype(np.float32))
    tfidf_chunk_files.append(fname)

print("TF-IDF transformation complete!")

TF-IDF transform: 100%|██████████| 27/27 [09:20<00:00, 20.76s/it]

TF-IDF transformation complete!


In [46]:
# stratified sampling for chi sqaure - supervised feature selection

from sklearn.model_selection import train_test_split

# sample only 5% of the dataset - 5% of data ~ 130k
X_sample, _, y_sample, _ = train_test_split(
    reviews_cleaned['review'],
    reviews_cleaned['sentiment'],
    stratify=reviews_cleaned['sentiment'],
    test_size=0.95,
    random_state=42
)

X_sample_tfidf = tfidf.transform(X_sample)
print("TF-IDF transformation on sample completed!")


TF-IDF transformation on sample completed!


In [47]:
# apply chi square

from sklearn.feature_selection import SelectKBest, chi2

FINAL_K = 20_000

selector = SelectKBest(chi2, k=FINAL_K)
selector.fit(X_sample_tfidf, y_sample)
selected_indices = selector.get_support(indices=True)
final_features = reduced_vocab[selected_indices]

np.save(PROJECT_PATH + "final_vocab.npy", final_features)
joblib.dump(selector, PROJECT_PATH + "chi2_selector.pkl")
print("Chi square feature selection completed. Final features:", len(final_features))

Chi square feature selection completed. Final features: 20000


In [48]:
# reduce full TF-IDF chunks to chi-square-selected features

from scipy import sparse
from scipy.sparse import save_npz, vstack, load_npz
from tqdm import tqdm

final_chunk_files = []

for fname in tqdm(tfidf_chunk_files, desc="Applying final features to all chunks"):
    X = sparse.load_npz(fname)
    X_selected = X[:, selected_indices]
    out_fname = fname.replace("tfidf_chunk_", "final_chunk_")
    save_npz(out_fname, X_selected)
    final_chunk_files.append(out_fname)

Applying final features to all chunks: 100%|██████████| 27/27 [02:33<00:00,  5.67s/it]


In [ ]:
# after last step, each chunk file contains (100k samples) x (20k features) sparse TF-IDF matrix
# now ready for model training

In [ ]:
# Logistic Regression for 3-Class Problems
# 2 options:
# 1. Softmax Regression
# 2. One-vs-Rest: less accurate but simpler

In [8]:
# keep all the classes in a separate file - same order as the full dataset
import pandas as pd
import numpy as np
reviews_cleaned = pd.read_csv("/content/drive/MyDrive/EECS4412/data/project/reviews_cleaned.csv")
y_full = reviews_cleaned["sentiment"].astype(int).to_numpy()
np.save("/content/drive/MyDrive/EECS4412/data/project/y_full.npy", y_full)

print("y_full saved. Shape:", y_full.shape)

y_full saved. Shape: (2616702,)


In [9]:
import numpy as np
from scipy.sparse import load_npz, csr_matrix
import os, glob, math, time
from tqdm import tqdm

# configs
PROJECT_PATH = "/content/drive/MyDrive/EECS4412/data/project/"
FINAL_CHUNKS_GLOB = PROJECT_PATH + "final_chunk_*.npz"
Y_FILE = PROJECT_PATH + "y_full.npy"
FINAL_VOCAB_FILE = PROJECT_PATH + "final_vocab.npy"

# load final vocab
final_vocab = np.load(FINAL_VOCAB_FILE, allow_pickle=True)
D = len(final_vocab)
print("D (feature dimension) =", D)

y_full = np.load(Y_FILE)   # shape (N,)
N = y_full.shape[0]
print("Total training samples:", N)

# precompute mapping of chunk files in sorted order
chunk_files = sorted(glob.glob(FINAL_CHUNKS_GLOB))
if len(chunk_files) == 0:
    raise RuntimeError("No chunk files found: " + FINAL_CHUNKS_GLOB)

chunk_row_counts = []
for fname in chunk_files:
    tfidfMtrix_temp = load_npz(fname)
    chunk_row_counts.append(tfidfMtrix_temp.shape[0])
    del tfidfMtrix_temp

# cumulative row indexing
chum_rows = np.cumsum([0] + chunk_row_counts)
print("Found", len(chunk_files), "chunks.")

D (feature dimension) = 20000
Total training samples: 2616702
Found 27 chunks.


In [14]:
# hyper-params
import re

NUM_CLASSES = 3
EPOCHS = 5
BATCH_SIZE = 2048        # minibatch rows (tune to RAM)
LR = 0.5                 # initial learning rate; tune (e.g. 0.1 - 1.0)
L2 = 1e-4                # regularization strength
MOMENTUM = 0.9           # 0 for none
SEED = 42
DTYPE = np.float32

use_adam = False

# utils
def softmax_rows(z):
    z_max = np.max(z, axis=1, keepdims=True)
    e = np.exp(z - z_max) # subtract max for stability
    s = np.sum(e, axis=1, keepdims=True)
    return e / s

# def labels_for_chunk(i):
#     start = chum_rows[i]
#     end = chum_rows[i+1]
#     return y_full[start:end]
def get_labels_from_filename(fname, y_full):
    m = re.search(r"(\d+)_(\d+)\.npz$", fname)
    if m is None:
        raise ValueError(f"Chunk filename format incorrect: {fname}")
    start = int(m.group(1))
    end = int(m.group(2))
    return y_full[start:end]

def one_hot(y, k):
    n = y.shape[0]
    oh = np.zeros((n, k), dtype=DTYPE)
    oh[np.arange(n), y] = 1.0
    return oh

In [15]:
# initialize Logisitc Regression model

rng = np.random.default_rng(SEED)

W = rng.normal(0, 0.01, size=(D, NUM_CLASSES)).astype(DTYPE)
b = np.zeros(NUM_CLASSES, dtype=DTYPE)

velocity_W = np.zeros_like(W)
velocity_b = np.zeros_like(b)

if use_adam:
    mW = np.zeros_like(W)
    vW = np.zeros_like(W)
    mb = np.zeros_like(b)
    vb = np.zeros_like(b)
    beta1, beta2 = 0.9, 0.999
    eps = 1e-8
    t_step = 0

In [16]:
# train
import gc
for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    epoch_loss = 0
    seen = 0

    # shuffle chunk order each epoch
    rng.shuffle(chunk_files)

    # iterate chunks in order
    for i, fname in enumerate(tqdm(chunk_files, desc=f"Epoch {epoch}")):
        X_chunk = load_npz(fname).tocsr()
        # y_chunk = labels_for_chunk(i)
        y_chunk = get_labels_from_filename(fname, y_full)
        print(i, X_chunk.shape[0], y_chunk.shape[0])

        m = X_chunk.shape[0]
        seen += m

        idx = np.arange(m)
        rng.shuffle(idx)
        X_chunk = X_chunk[idx]
        y_chunk = y_chunk[idx]

        # minibatch within chunk
        for start in range(0, m, BATCH_SIZE):
            end = min(start + BATCH_SIZE, m)
            Xb = X_chunk[start:end] # sparse CSR (nb, D)
            yb = y_chunk[start:end] # (nb,)
            nb = yb.shape[0]

            # compute scores z = Xb.dot(W) + b -> (nb, K)
            z = Xb.dot(W) + b # dense (nb, K)

            # probabilities
            p = softmax_rows(z) # (nb, K)

            # cross-entropy, loss
            loss = -np.log(p[np.arange(nb), yb] + 1e-12).sum()
            epoch_loss += loss

            # gradients
            # gradient of loss w.r.t. z: G = p; G[range(nb), y] -= 1
            G = p
            G[np.arange(nb), yb] -= 1
            G /= nb #  average over batch

            # gradient w.r.t. W: dW = Xb.T @ G + L2 * W
            # Xb.T @ G  -> (D, K) dense
            dW = Xb.T.dot(G) + L2 * W
            db = G.sum(axis=0) # (K,)

            # SGD update using momentum
            velocity_W = MOMENTUM * velocity_W - LR * dW
            W += velocity_W

            velocity_b = MOMENTUM * velocity_b - LR * db
            b += velocity_b

        # delete chunk after all minibatches processed to free up memory
        del X_chunk
        del y_chunk
        gc.collect()

    print(f"Epoch {epoch}: avg loss = {epoch_loss/seen:.6f}, time = {time.time()-t0:.1f}s")

    # save checkpoint
    np.save(PROJECT_PATH + f"W_epoch{epoch}.npy", W)
    np.save(PROJECT_PATH + f"b_epoch{epoch}.npy", b)
    print("Checkpoint saved.")

Epoch 1:   0%|          | 0/27 [00:00<?, ?it/s]

0 100000 100000


Epoch 1:   4%|▎         | 1/27 [00:00<00:16,  1.57it/s]

1 100000 100000


Epoch 1:   7%|▋         | 2/27 [00:01<00:15,  1.63it/s]

...

25 100000 100000


Epoch 5:  96%|█████████▋| 26/27 [00:18<00:00,  1.29it/s]

26 100000 100000


Epoch 5: 100%|██████████| 27/27 [00:18<00:00,  1.43it/s]

Epoch 5: avg loss = 0.450430, time = 18.9s
Checkpoint saved.


In [ ]:
# use a small representative sample for hyperparameter tuning (e.g., 5–10% of the dataset).
# train the model from scratch on that sample for a few epochs.
# keep a small separate test set to evaluate model after tuning.
# monitor metrics: validation accuracy, cross-entropy loss.
# select best hyperparameters and then scale up to full dataset.
# solution: possibly use a tuning loop with different hyperparams

In [18]:
# stratified sample

from sklearn.model_selection import train_test_split

SAMPLE_RATIO = 0.05
MAX_SAMPLE = 150_000
N = len(reviews_cleaned)

sample_size = min(int(N * SAMPLE_RATIO), MAX_SAMPLE)
print("Target sample size =", sample_size)

# create stratified sample
_, sample_idx = train_test_split(
    np.arange(N),
    test_size=sample_size,
    stratify=y_full,
    random_state=42
)

sample_idx = np.sort(sample_idx)
print("Sampled rows =", len(sample_idx))

Target sample size = 130835
Sampled rows = 130835


In [19]:
# extract TF-IDF rows from chunk files - load only the required rows

from scipy.sparse import vstack, csr_matrix

sample_matrices = []
current_start = 0

itr = 0

for fname in tqdm(chunk_files, desc="Extracting sample rows"):
    X_chunk = load_npz(fname).tocsr()
    m = X_chunk.shape[0]

    # rows of sample_idx that belong to this chunk
    mask = (sample_idx >= current_start) & (sample_idx < current_start + m)
    local_rows = sample_idx[mask] - current_start

    if len(local_rows) > 0:
        sample_matrices.append(X_chunk[local_rows])

    current_start += m

# combine
X_sample = vstack(sample_matrices).tocsr()
y_sample = y_full[sample_idx]

print("X_sample shape =", X_sample.shape)
print("y_sample shape =", y_sample.shape)

# save sample for reuse
from scipy.sparse import save_npz
save_npz(PROJECT_PATH + "X_sample.npz", X_sample)
np.save(PROJECT_PATH + "y_sample.npy", y_sample)

print("Sample saved!")

Extracting sample rows: 100%|██████████| 27/27 [00:10<00:00,  2.53it/s]


X_sample shape = (130835, 20000)
y_sample shape = (130835,)
Sample saved!


In [20]:
# split sample into train/validation

X_sample = load_npz(PROJECT_PATH + "X_sample.npz")
y_sample = np.load(PROJECT_PATH + "y_sample.npy")

X_train, X_val, y_train, y_val = train_test_split(
    X_sample, y_sample,
    test_size=0.2,
    stratify=y_sample,
    random_state=42
)

print("Train =", X_train.shape, "Val =", X_val.shape)

Train = (104668, 20000) Val = (26167, 20000)


In [21]:
# initialize params for tuning

def init_params(D, K, seed=42):
    rng = np.random.default_rng(seed)
    W = rng.normal(0, 0.01, size=(D, K))
    b = np.zeros(K)
    return W, b

In [39]:
# operates only on X_train / y_train using SGD + momentum

import time
from sklearn.metrics import f1_score, accuracy_score

# cfg: dict containing hyperparameters
def train_one_config(cfg, X_train, y_train, X_val, y_val, verbose=False):
    LR = cfg['lr']
    BATCH = cfg['batch']
    L2 = cfg['l2']
    OPT = cfg['opt']         # 'adam' or 'sgd'
    MOM = cfg['momentum']
    EPOCHS = cfg['epochs']

    K = 3  # 3 sentiment classes
    W, b = init_params(D, K, seed=cfg['seed'])

    # initialize velocities for SGD + momentum
    velocity_W = np.zeros_like(W)
    velocity_b = np.zeros_like(b)

    # Adam parameters
    # mW = np.zeros_like(W)
    # vW = np.zeros_like(W)
    # mb = np.zeros_like(b)
    # vb = np.zeros_like(b)
    # beta1 = 0.9
    # beta2 = 0.999
    # eps = 1e-8
    # t = 0

    history = []
    best_val_loss = float("inf")
    patience = cfg.get("patience", 1)
    patience_counter = 0

    for ep in range(EPOCHS):
        t0 = time.time()

        idx = np.arange(len(y_train))
        np.random.shuffle(idx)

        Xtr = X_train[idx]
        ytr = y_train[idx]

        losses = []
        # minibatches
        for start in range(0, len(ytr), BATCH):
            end = min(start + BATCH, len(ytr))
            Xb = Xtr[start:end]
            yb = ytr[start:end]
            nb = len(yb)

            # forward
            z = Xb.dot(W) + b
            p = softmax_rows(z)
            loss = -np.log(p[np.arange(nb), yb] + 1e-12).mean()
            losses.append(loss)

            # gradients
            G = p
            G[np.arange(nb), yb] -= 1
            G /= nb

            dW = Xb.T.dot(G) + L2 * W
            db = G.sum(axis=0)

            # update
            if OPT == "sgd":
                velocity_W = MOM * velocity_W - LR * dW
                W += velocity_W

                velocity_b = MOM * velocity_b - LR * db
                b += velocity_b

            # else:  # ADAM
            #     t += 1
            #     mW = beta1*mW + (1-beta1)*dW
            #     vW = beta2*vW + (1-beta2)*(dW*dW)
            #     mb = beta1*mb + (1-beta1)*db
            #     vb = beta2*vb + (1-beta2)*(db*db)

            #     mW_hat = mW / (1 - beta1**t)
            #     vW_hat = vW / (1 - beta2**t)
            #     mb_hat = mb / (1 - beta1**t)
            #     vb_hat = vb / (1 - beta2**t)

            #     W -= LR * mW_hat / (np.sqrt(vW_hat) + eps)
            #     b -= LR * mb_hat / (np.sqrt(vb_hat) + eps)

        # compute val metrics
        z_val = X_val.dot(W) + b
        p_val = softmax_rows(z_val)
        val_loss = -np.log(p_val[np.arange(len(y_val)), y_val] + 1e-12).mean()
        y_pred = p_val.argmax(axis=1)
        val_f1 = f1_score(y_val, y_pred, average='macro')
        val_acc = accuracy_score(y_val, y_pred)
        history.append((val_loss, val_f1, val_acc))

        if verbose:
            print(f"Epoch {ep+1}: ValLoss={val_loss:.4f}, ValF1={val_f1:.4f}, ValAcc={val_acc:.4f}, time={time.time()-t0:.1f}s")

        # early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter > patience:
                if verbose:
                    print("Early stopping triggered.")
                break

    return {
        'val_loss': best_val_loss,
        'val_f1': val_f1,
        "val_acc": val_acc,
        'W': W,
        'b': b,
        'history': history
    }

In [36]:
# define search space and random search loop

import random

def sample_hyperparams():
    return {
        "opt": "sgd",
        "lr": random.choice([1e-4, 3e-4, 1e-3, 3e-3, 1e-2]),
        "l2": random.choice([1e-5, 1e-4, 1e-3, 1e-2]),
        "batch": random.choice([1024, 2048, 4096]),
        "momentum": random.choice([0.0, 0.8, 0.9]),
        "epochs": 3,
        "seed": random.randint(1, 10_000),
        "patience": 1
    }

In [41]:
# run random search
import joblib

NUM_TRIALS = 30
results = []

for t in range(NUM_TRIALS):
    print(f"\n Trial {t+1}/{NUM_TRIALS}")
    cfg = sample_hyperparams()

    # velocities are runtime satate, no need to save
    save_cfg = {
        k: v for k, v in cfg.items()
        if k not in ["velocity_W", "velocity_b"]
    }

    res = train_one_config(
        cfg,
        X_train, y_train,
        X_val, y_val,
        verbose=False
    )

    entry = {
        "trial": t,
        "cfg": cfg,
        "val_f1": res['val_f1'],
        "val_acc": res['val_acc'],
        "val_loss": res['val_loss']
    }
    results.append(entry)
    print(res)

# save results
joblib.dump(results, PROJECT_PATH + "tuning_results.pkl")

print("Tuning completed!")


 Trial 1/30
{'val_loss': np.float64(0.9173443604900238), 'val_f1': 0.29533763312685807, 'val_acc': 0.7953529254404402, 'W': array([[-0.01064106,  0.00130162,  0.00276655],
       [-0.01066395,  0.00863957, -0.00470989],
       [-0.00751118,  0.00184958, -0.00445932],
       ...,
       [ 0.00272476, -0.00111307, -0.00151201],
       [-0.01316521, -0.01370441,  0.00596549],
       [-0.02033492,  0.00259929,  0.00126096]]), 'b': array([-0.15580136,  0.29026917, -0.13446781]), 'history': [(np.float64(1.0445691050805932), 0.29533763312685807, 0.7953529254404402), (np.float64(0.9756439219520336), 0.29533763312685807, 0.7953529254404402), (np.float64(0.9173443604900238), 0.29533763312685807, 0.7953529254404402)]}

 Trial 2/30
{'val_loss': np.float64(0.97098252907474), 'val_f1': 0.29533763312685807, 'val_acc': 0.7953529254404402, 'W': array([[-0.00696904,  0.00642056, -0.00222787],
       [ 0.00244574,  0.00714857,  0.01484885],
       [ 0.00482186,  0.00317512,  0.00894493],
       ...,
   

In [42]:
# best hyperparams
import joblib

results = joblib.load(PROJECT_PATH + "tuning_results.pkl")

best = max(results, key=lambda x: x['val_f1'])

print("Best trial configuration:")
print(best)

best_cfg = best["cfg"]

Best trial configuration:
{'trial': 0, 'cfg': {'opt': 'sgd', 'lr': 0.001, 'l2': 0.01, 'batch': 4096, 'momentum': 0.9, 'epochs': 3, 'seed': 6832, 'patience': 1}, 'val_f1': 0.29533763312685807, 'val_acc': 0.7953529254404402, 'val_loss': np.float64(0.9173443604900238)}
